In [1]:
import utils

## Day 8: Handheld Halting

[#](https://adventofcode.com/2020/day/8). We make a mini computer which can process instructions in the form `acc +4` 

- `acc` adds to a global value.
- `jmp` jumps to a new instruction relative to itself. `jmp +2` skips 1 instruction, `jmp +` goes to the next.
- `nop` does nothing.

In [2]:
test8 = """
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
""".strip().splitlines()

inp8 = utils.get_input(8, splitlines=True)

I first wrote this as a func, but using a class here made more sense, as a computer is an object with state, so its nicer to have all its stuff contained inside a class.

In [3]:
class Computer:
    def __init__(self, inp):
        self.accumulator = 0
        self.idx = 0
        self.instructions = inp
        self.terminate = len(inp)
        
    def __str__(self):
        return f"idx: {self.idx:2} acc:{self.accumulator:2} - {self.instructions[self.idx]}"
    
    def get_ins(self):
        ins, val = self.instructions[self.idx].split(" ")
        val = int(val)
        return ins, val
    
    def step(self):
        """performs 1 step"""
        ins, val = self.get_ins()
        #print(self)
        if ins == "acc":
            self.accumulator += val
            self.idx += 1
        elif ins == "jmp":
            self.idx += val
        elif ins == "nop":
            self.idx += 1
            
    def run_once(self):
        seen = []
        while self.idx not in seen:
            seen.append(self.idx)
            self.step()
        print(self)
        return self.accumulator
        
comp = Computer(inp=test8)
assert comp.run_once() == 5

comp = Computer(inp=inp8)
comp.run_once()

idx:  1 acc: 5 - acc +1
idx: 469 acc:1859 - jmp -159


1859

## Part 2

We have a corrupted instruction somewhere causing a infinite loop.

Either a `jmp` should be a `nop` or vice versa. The program should terminate when it tries to execute an instruction after the last one.

I updated the computer class above to have a self.terminate - if the computer reaches that then it should terminate. The fix instructions is outside the class, since I am making a new computer for each set of new instructions.

The below brute forces the solution, though there must be a more elegant way of doing this.

In [5]:
def fix_instructions(inp=test8):
    for i, line in enumerate(inp):
        
        # flip the bad instructions here
        if line[:3] == "jmp":
            line = "nop" + line[3:]
        elif line[:3] == "nop":
            line = "jmp" + line[3:]
        
        # make a new set of changed instructions
        new_instructions = inp[:]
        new_instructions[i] = line
        
        comp = Computer(new_instructions)
        
        # run the comp till it excecutes 3x or reaches the terminator
        for _ in range(len(inp)*3): # randomly choose 3
            comp.step()
            if comp.idx == comp.terminate:
                return comp.accumulator

assert fix_instructions(test8) == 8
fix_instructions(inp8)

1235